### Wednesday, December 27, 2023

Trying this again ... using the hfpt_Dec14 container. 

I am going to torch all the models in the /root/.cache/huggingface/hub directory to see how much space gets freed. 

Data2 is at 355.3 GB currently ... now let's clear away all the current models and see what we gain ... hmm odd, nothing seems to have been gained ... sigh.

Right .... have to run 

1) pip install auto-gptq 
2) pip install optimum


### Friday, November 17, 2023

Been doing some model cleanup on the hfpt_Oct28 container, and want to re-run this just to ensure all is still well ...

Yup! This still works! Nice!

### Thursday, November 16, 2023

https://huggingface.co/TheBloke/CodeLlama-34B-Instruct-GPTQ

Let's see if we can get this to run locally, shall we ... 

I am trying this model because it has the most downloads of all his models, AND of all the huggingface models that are 'code' related.

Wow! This actually works on the 4090! I am stunned! 20524MiB of VRAM!


In [5]:
!ls /root/.cache/huggingface/hub

In [4]:
!mkdir /root/.cache/huggingface/hub

In [2]:
!rm -rf /root/.cache/huggingface/hub

In [ ]:
# docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ c8324b70601d://root/.cache/huggingface/hub
# Successfully copied 18.3GB to c8324b70601d://root/.cache/huggingface/hub

In [1]:
!ls /root/.cache/huggingface/hub

models--TheBloke--CodeLlama-34B-Instruct-GPTQ  version.txt


In [ ]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 18.3GB to /home/rob/Data3/huggingface/transformers


# (base) rob@KAUWITB:~$ docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 18.3GB to c9b676310ea0://home/rob/Data2/huggingface/transformers
# (base) rob@KAUWITB:~$ 


In [1]:
# You need this to just target the 4090, othewise the 34B model does NOT load.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# !ls /home/rob/Data2/huggingface/transformers

models--TheBloke--CodeLlama-34B-Instruct-GPTQ  version.txt


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [3]:
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

# 15.1s 18740 MiB VRAM ... why less than before??

# 16.7s to load, takes up 20132Mib VRAM

In [4]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# # Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     top_k=40,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])

# 6.1s 18856 MiB VRAM

# 22.5s 20524Mib VRAM




*** Generate:
<s> [INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

```
def solve(n):
    # Write your code here
    return "AI is a rapidly growing field that holds great promise for the future. It has many applications in various industries such as healthcare, finance, education, and transportation."


# Test cases
def test_solve():
    assert solve(1) == "AI is a rapidly growing field that holds great promise for the future. It has many applications in various industries such as healthcare, finance, education, and transportation."


# Run test cases
test_solve()
```</s>


In [5]:
# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

# 7.2s 18878 MiB VRAM

# 11.9s 20524Mib VRAM

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

```
def tell_me_about_AI(topic):
    if topic == "":
        return "I can tell you about many topics related to AI, such as machine learning, natural language processing, computer vision, and robotics. If you have a specific question or topic in mind, please let me know and I'll do my best to provide a detailed response."
    else:
        return f"I'm not able to provide information on the topic of '{topic}'. However, I can try to help you find more information on this topic by providing suggestions for books, articles, or other resources that may be relevant."

# Example usage:
print(tell_me_about_AI(""))
print(tell_me_about_AI("machine learning"))
```
